##greedy-fragment-partitioner

Partitions reads greedily based on hapcut blocks and adds the information to a BAM file, also produces a statistics file of blocks and phasable and unphasable reads. Does some other things too like partitioning based on the truth trio data (see code)

In [1]:
%matplotlib inline
%run ./ipy_setup.py

In [2]:
# %load ./ipy_setup.py
# Imports / style (run this first always)

# %matplotlib inline ## magic functions don't work here!
from collections import defaultdict
import json
import sys
import time

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib import rcParams
import matplotlib.cm as cm
import matplotlib as mpl

class AwesomeError(Exception):
     def __init__(self, value):
         self.value = value
         pass
     def __str__(self):
         return repr(self.value)
         pass

#colorbrewer2 Dark2 qualitative color table
dark2_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
                (0.8509803921568627, 0.37254901960784315, 0.00784313725490196),
                (0.4588235294117647, 0.4392156862745098, 0.7019607843137254),
                (0.9058823529411765, 0.1607843137254902, 0.5411764705882353),
                (0.4, 0.6509803921568628, 0.11764705882352941),
                (0.9019607843137255, 0.6705882352941176, 0.00784313725490196),
                (0.6509803921568628, 0.4627450980392157, 0.11372549019607843)]

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 200
rcParams['axes.color_cycle'] = dark2_colors
rcParams['lines.linewidth'] = 2
rcParams['axes.facecolor'] = 'white'
rcParams['font.size'] = 14
rcParams['patch.edgecolor'] = 'white'
rcParams['patch.facecolor'] = dark2_colors[0]
rcParams['font.family'] = 'StixGeneral'


def remove_border(axes=None, top=False, right=False, left=True, bottom=True):
    """
    Minimize chartjunk by stripping out unnecesasry plot borders and axis ticks
    
    The top/right/left/bottom keywords toggle whether the corresponding plot border is drawn
    """
    ax = axes or plt.gca()
    ax.spines['top'].set_visible(top)
    ax.spines['right'].set_visible(right)
    ax.spines['left'].set_visible(left)
    ax.spines['bottom'].set_visible(bottom)
    
    #turn off all ticks
    ax.yaxis.set_ticks_position('none')
    ax.xaxis.set_ticks_position('none')
    
    #now re-enable visibles
    if top:
        ax.xaxis.tick_top()
    if bottom:
        ax.xaxis.tick_bottom()
    if left:
        ax.yaxis.tick_left()
    if right:
        ax.yaxis.tick_right()
        
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

import Bio as bp
from Bio.Sequencing.Applications import BwaAlignCommandline as bwa_aln
from Bio.Sequencing.Applications import BwaSamseCommandline as bwa_samse
from Bio.Sequencing.Applications import BwaSampeCommandline as bwa_sampe
from Bio.Sequencing.Applications import BwaIndexCommandline as bwa_index
from Bio.Sequencing.Applications import BwaBwaswCommandline as bwa_bwasw
import HTSeq as ht
import subprocess

# progress bar

import sys, time
try:
    from IPython.display import clear_output
    have_ipython = True
except ImportError:
    have_ipython = False

class ProgressBar:
    def __init__(self, iterations):
        self.iterations = iterations
        self.start = time.time()
        self.time = time.time()
        self.count = 0
        self.prog_bar = '[]'
        self.fill_char = '*'
        self.width = 40
        self.__update_amount(0)
        if have_ipython:
            self.animate = self.animate_ipython
        else:
            self.animate = None

    def animate_ipython(self):
        self.count += 1
        if round(time.time() - self.time) > 1: # throttling to 1 second
            print '\r', self,
            sys.stdout.flush()
            self.update_iteration(self.count)
            self.time = time.time()

    def update_iteration(self, elapsed_iter):
        elapsed = time.time()-self.start # in seconds
        time_remaining = (self.iterations - elapsed_iter)*elapsed/float(elapsed_iter)
        self.__update_amount((elapsed_iter / float(self.iterations)) * 100.0)
        self.prog_bar += '  %d of %s complete' % (elapsed_iter, self.iterations)
        self.prog_bar += ' Elapsed: %s minutes\t Remaining: %s minutes' % (round(elapsed/60.,1), round(time_remaining/60.,1))

    def __update_amount(self, new_amount):
        percent_done = int(round((new_amount / 100.0) * 100.0))
        all_full = self.width - 2
        num_hashes = int(round((percent_done / 100.0) * all_full))
        self.prog_bar = '[' + self.fill_char * num_hashes + ' ' * (all_full - num_hashes) + ']'
        pct_place = (len(self.prog_bar) // 2) - len(str(percent_done))
        pct_string = '%d%%' % percent_done
        self.prog_bar = self.prog_bar[0:pct_place] + \
            (pct_string + self.prog_bar[pct_place + len(pct_string):])

    def __str__(self):
        return str(self.prog_bar)



In [31]:
#!/usr/bin/env python

# imports
import os, sys, getopt
import pysam
from itertools import groupby
from operator import itemgetter
import pandas as pd
import numpy as np

global count
count = 0

# init main
def main(argv):
    hairs_file = ''
    hapcut_file = ''
    bam_file = ''
    out_file = ''
    help = 'greedy_partitioner.py -h <input.hairs> -c <input.hapcut> -i <input.bam> -o <output.ann.bam>'
    try:
        opts, args = getopt.getopt(argv,"h:c:i:o:",["hairs=","hapcut=", "input=", "output="])
    except getopt.GetoptError:
        print help
        sys.exit(2)
    for opt, arg in opts:
        if opt == '--help':
            print help
            sys.exit()
        elif opt in ("-h", "--hairs"):
            hairs_file = arg
        elif opt in ("-c", "--hapcut"):
            hapcut_file = arg
        elif opt in ("-i", "--input"):
            bam_file = arg
        elif opt in ("-o", "--output"):
            out_file = arg
        else:
            assert False, "unhandled option"

    assert pysam.Samfile(bam_file, 'rb'), 'ERROR: Cannot open bam file for reading.'
    assert open(bam_file + '.bai', 'rb'), 'ERROR: bam file is not indexed!'
    bam_fp = pysam.Samfile(bam_file, 'rb')

    if out_file==None:
        out_file = bam_file + ".ann_haplotypes_" + time.strftime("%m%d%y_%H%M%S") + '.bam'

    assert pysam.AlignmentFile(out_file, "wb", template=bam_fp), 'ERROR: Cannot open output file for writing.'
    out_fp = pysam.AlignmentFile(out_file, "wb", template=bam_fp)

    assert open(hairs_file), 'ERROR: Cannot access hairs file.'
    assert open(hapcut_file), 'ERROR: Cannot open hapcut file.'

    hair_reader = HairReader(hairs_file)
    block_reader = HapCutReader(hapcut_file)
    stats_file = out_file + ".interblock_stats.tsv"
    sys.stdout.write("Loaded greedy_partitoner.py, beginning execution. \n")
    sys.stdout.write("Placing hairfile reads into haplotype blocks. \n")
    sys.stdout.flush()
    hair_reader.assemble_blocks(block_reader)
    sys.stdout.write("Placing bamfile reads into haplotype blocks. \n")
    sys.stdout.flush()
    unphased_reads = unphased_heuristics(hair_reader, block_reader, bam_fp)
    sys.stdout.write("Partitioning reads. \n")
    sys.stdout.flush()
    tag_reads(bam_fp, hair_reader, block_reader, unphased_reads, out_fp) #begin tagging reads
    sys.stdout.write("Calculating interblock statistics. \n")
    sys.stdout.flush()
    interblock_stats(hair_reader, block_reader, stats_file, bam_fp) #generate interblock stats
    bam_fp.close()
    out_fp.close()

# end of main

### CLASSES ###

class BlockVariant:
    def __init__ (self, variantline):
        # variant_id haplotype_1 haplotype_2 chromosome position refallele variantallele genotype allele_counts:genotype_likelihoods:delta:MEC_variant
        ll = variantline.strip().split("\t")
        var_id, hap1, hap2, chrom, pos, r_allele, v_allele, genotype, info_str = ll
        self.chrom, self.r_allele, self.v_allele, self.info_str = chrom, r_allele, v_allele, info_str
        self.var_id, self.hap1, self.hap2, self.pos = int(var_id), hap1, hap2, int(pos)
        allele_counts, genotype_likelihoods, delta, MEC_variant = info_str.split(":")[0:4]
        self.ref_count, self.alt_count = map(int, allele_counts.split(","))
        gen_00, gen_01, gen_11 = map(float, genotype_likelihoods.split(","))
        self.gen_like = {"0/0":gen_00, "0/1":gen_01, "1/1":gen_11}
        self.delta = float(delta)
        self.MEC_variant = MEC_variant
    def __repr__ (self):
        return "<BlockVariant, var_id: %s>" % str(self.var_id)


class Block: # part of block reader
    def __init__ (self, blockline):
        # "BLOCK: offset:" first_variant_block "len:" length_of_block "phased": phased_variants_block SPAN: 
        # lengthspanned MECscore score fragments #fragments

        ll               = blockline.strip().split()
        self.offset      = int(ll[2])
        self.total_len   = int(ll[4])
        self.phased      = int(ll[6])
        self.span        = int(ll[8])
        self.MECscore    = float(ll[10])
        self.fragments   = int(ll[12])
        self.variants 	 = dict() # default to empty
        self.variant_ids = set()
        self.chrom = None
        self.start = None
        self.end = None
        self.informative_reads = []
        self.unphased_reads = []
        self.unphased_read_set = set()
        self.read_count = 0
        self.read_set = set()

    def __repr__ (self):
        return "<Block, offset_id: %s>" % str(self.offset)

    def addVariant(self, variantline):
        variant = BlockVariant(variantline)
        self.variants[variant.var_id] = variant
        self.variant_ids.add(variant.var_id)
        self.updatePosition(variant)

    def updatePosition(self, variant): # we need to do this because sometimes the variant isn't associated with a block
        if (self.chrom != None) & (self.start != None) & (self.end != None):
            if variant.pos < self.start:
                self.start = variant.pos
            elif variant.pos > self.end:
                self.end = variant.pos
            if self.chrom != variant.chrom:
                sys.stderr.write("WARNING: Cannot add variants from other contigs to current block.")
                return 1
        else:
            self.chrom = variant.chrom
            self.start = variant.pos
            self.end = variant.pos

    def addReadsToBlock(self, read_dict):
        self.informative_reads = []
        for k,read in read_dict.iteritems():
            read_ids = frozenset([var[0] for block in read.blocks for var in block])
            if len(set(read_ids).intersection(set(self.variant_ids))) > 0:
                self.informative_reads.append(read)
        self.read_count = len(self.informative_reads)
        self.read_set = frozenset([x.read_id for x in self.informative_reads])

    def concordance(self, input_reads):
        ''' this should return a dict of (#T,#F) tuples per variant
         each element is a variant's concordance with the reads
         using the read's haplotype information, we can establish whether the read's phasing
         is consistent with how the variant was phased '''
        variant_concord = dict()
        support_reads_hap2 = 0
        against_reads_hap2 = 0
        support_reads_hap1 = 0
        against_reads_hap1 = 0
        for k,variant in self.variants.iteritems():
            for read in input_reads:
                if variant.var_id in read.positions:
                    read_allele = read.alleles[read.positions.index(variant.var_id)]
                    hapstate = read.haplotypes[self.offset]
                    if hapstate == 2:
                        if variant.hap2 != read_allele:
                            against_reads_hap2 += 1
                        else:
                            support_reads_hap2 += 1
                    else:
                        if variant.hap1 != read_allele:
                            against_reads_hap1 += 1
                        else:
                            support_reads_hap1 += 1
        variant_concord[self.offset] = {"hap1": (support_reads_hap1, against_reads_hap1), 
                                               "hap2": (support_reads_hap2, against_reads_hap2)}
        return variant_concord

    def variant(self, var_id):
        try:
            return self.variants[var_id]
        except:
            return None

    def interblock_reads(self, input_reads):
        out_reads = []
        for read in input_reads:
            if read.read_id not in self.read_set:
                if read.chrom == self.chrom:
                    if ((read.start < self.end) & (read.end > self.end)) | \
                        ((read.end > self.start) & (read.start < self.start)) | \
                        ((read.end <= self.end) & (read.start >= self.start)):
                        out_reads.append(read)
        return out_reads
    
    def add_read(self, read_obj):
        if read_obj.read_id not in self.read_set:
            self.informative_reads.append(read_obj)
            self.read_set.add(read_obj.read_id)
    
    def add_unphased_reads(self, bam_fp):
        reads = bam_fp.fetch(region=self.chrom + ':' + str(self.start) + '-' + str(self.end))
        for read in reads:
            if read.query_name not in self.read_set:
                self.unphased_reads.append(read)
                self.unphased_read_set.add(read.query_name)
        return None

class HapCutReader: # hapcut file reader

    def __init__ ( self, fn ):
        self.fn = fn
        self.blocks = dict()
        self.translate = dict()
        for block in self.read_file_to_blocks(fn):
            self.blocks[block.offset] = block
            for v in block.variant_ids: # v is an id
                self.translate[v] = block.offset

    def loc(self, block_id):
        try:
            return self.blocks[self.translate[block_id]]
        except:
            return None

    def read_file_to_blocks(self, fn):
        with open(fn) as f:
            currBlock = None
            for l in f:
                if l[0] == "B": # starting a new block
                    currBlock = Block(l)
                elif l[0] == "*": # ending a block
                    yield currBlock
                else:
                    currBlock.addVariant(l)

    def __repr__ (self):
        return "<HapCutReader, filename: %s>" % self.fn
    
    def assemble_reads(self, hair_reader):
        for k,block in self.blocks.iteritems():
            block.addReadsToBlock(hair_reader.reads)

class HapCutRead: #hair file line

    def __init__ (self, hairline):
        #Column 1 is the number of consecutive set of SNPs covered by the fragment, NOT haplotype blocks.
        #Column 2 is the fragment id. 
        #Column 3 is the offset of the first block of SNPs covered by the fragment followed by the alleles at the SNPs in this block.
        #Column 5 is the offset of the second block of SNPs covered by the fragment followed by the alleles at the SNPs in this block.
        #...
        #The last column is a string with the quality values (Sanger fastq format) for all the alleles covered by the fragment (concatenated for all blocks). 
        #For example, if a read/fragment covers SNPs 2,3 and 5 with the alleles 0, 1 and 0 respectively, then the input will be:
        #2 read_id 2 01 5 0 AAC
        #Here AAC is the string corresponding to the quality values at the three alleles. The encoding of 0/1 is arbitrary but following the VCF format, 0 is reference and 1 is alternate. 
        hairlist = hairline.strip().split()
        self.blockcount = 0                # this information must be determined afterwards 
        self.read_id    = hairlist[1]      # read_id
        self.blocks     = []		       # an array of tuples corresponding to blocks
        self.positions  = []
        self.alleles    = []
        self.chrom      = None
        self.start      = None
        self.end        = None
        self.haplotypes = dict()             # an array of {"block_offset":"haplotype"} 
                                           # after partitioning
        for i in range(2, len(hairlist)-1, 2):
            position = int(hairlist[i])
            allele = hairlist[i+1]
            block = zip(range(position, position+len(allele)), allele)
            self.blocks.append(block)
            self.positions.extend(range(position, position+len(allele)))
            self.alleles.extend(allele)
            self.qualities  = hairlist[-1]         # a matched arary of the qualities of allele calls

    def __repr__(self):
        return "<HapCutRead, read_id: %s>" % str(self.read_id)

    def haplotype_fields(self):
        haps = ";".join([','.join([str(key), str(self.haplotypes[key])]) for key in self.haplotypes])
        haptag = [("ZH", haps), ("ZB", int(self.blockcount)), ("ZV", len(self.positions))]
        return haptag
    
    def addGenomicPositions(self, block_reader):
        arr = []
        chrom = None
        for position in self.positions:
            b = block_reader.loc(position)
            if b == None:
                continue
            if chrom == None:
                chrom = b.chrom
            arr.append(b.variant(position).pos)
        if len(arr) > 0:
            self.chrom = chrom
            self.start = np.min(arr)
            self.end = np.max(arr)
        else:
            self.chrom = '*'
            self.start = None
            self.end = None
    
    def assemble_blocks(self, block_reader):
        # it turns out that the blocks provided in a hapcut file don't actually correspond to real blocks
        # just contiguous alleles?
        self.blocks = []
        lastBlock = -1
        for ix, pos in enumerate(self.positions): # self.positions corresponds to a variant id
            currBlock = block_reader.loc(pos) # look up block associated with variant
            if currBlock == None: # if it's not, continue
                continue
            # let's also add ourselves to the block
            currBlock.add_read(self) #if we get a block back, add the read to the block's read set
            currBlock = currBlock.offset # set our read's block id
            if currBlock != lastBlock:
                self.blocks.append([])
            self.blocks[-1].append((pos, self.alleles[ix])) # a read can be more than one haplotype block long
            lastBlock = currBlock
        self.blockcount = len(self.blocks) # determine number of haplotype blocks read spans
        self.addGenomicPositions(block_reader) # determine start-end positions of read
    
class HairReader:

    def __init__ (self, fn):
        self.fn = fn
        self.reads = dict()
        with open (fn) as f:
            for l in f:
                newread = HapCutRead(l)
                self.reads[newread.read_id] = newread
        self.read_set = frozenset(self.reads.keys())

    def __repr__ (self):
        return "<HairReader, filename: %s>" % self.fn

    def loc(self, read_id):
        try:
            return self.reads[read_id]
        except:
            return None
    
    def assemble_blocks(self, block_reader):
        for k,read in self.reads.iteritems():
            read.assemble_blocks(block_reader)

### FUNCTIONS ###

'''

tag_reads()

Usage: Tags reads from a bam file corresponding to a particular haplotype, with haplotype
definitions from HapCut, under the optional tag "ZH".

Inputs:
    bam_fp
    A pysam.Samfile object pointing to the input file
    hair_reader
    A HairReader object pointing to the hairs file.
    block_reader
    A HapcutReader object pointing to the hapcut file.
    out_fp
    A pysam.AlignmentFile pointing to the output bam.
Outputs:
    (none - writes to out_fp)

'''

def tag_reads(bam_fp, hair_reader, block_reader, unphased_reads, out_fp):
    ''' tag_reads(bam_fp, hair_reader, block_reader, out_fp)'''
    global count
    for bamread in bam_fp.fetch(region="000000F", start = 10279323, end=10409440):
        count += 1
        if (count % 100) == 0:
            sys.stdout.write("\rWritten %s lines to output." % str(count))
            sys.stdout.flush()
        if bamread.query_name in hair_reader.read_set:
            read = hair_reader.loc(bamread.query_name)
            read = greedy_partition(read, block_reader)
            bamread.tags += read.haplotype_fields()      # add the haplotype information to hairfile read
            out_fp.write(bamread)
        elif bamread.query_name in unphased_reads:
            read = unphased_reads[bamread.query_name]
            read = greedy_partition(read, block_reader)
            bamread.tags += read.haplotype_fields()    # add the haplotype information to bam-based read
            out_fp.write(bamread)
        #out_fp.write(bamread) # write read to file

'''
greedy_partition()
Ryan Neff

inputs:
read
    a HapCutRead object
block_reader
    of the type HapCutReader

outputs:
    the original read, now with haplotype information.

translate hairfile alleles into blockvar IDs
get alleles in each read spanning blockvars
determine alleles for the two blocks from blockvar
partition read based on locally most probable alignment

'''

def greedy_partition(read, block_reader):
    for readblock in read.blocks:
        positions = [x[0] for x in readblock]
        alleles = [x[1] for x in readblock]
        allele_state = []
        offset = positions[0]
        hap = 0
        block = block_reader.loc(offset) #retrieve block the read is in
        if block == None: # this happens when hapcut throws out the block the read is in 
            continue
        offset = block.offset
        for ix, varpos in enumerate(positions):
            blockvar = block.variant(varpos) # retrieve variant from VCF file
            if blockvar.hap1 == alleles[ix]:
                allele_state.append(-1)
            elif blockvar.hap2 == alleles[ix]:
                allele_state.append(1)
            else:
                #sys.stderr.write("\nWarning: read allele matched no haplotypes.")
                #sys.stderr.write("\nHair read: %s" % read.read_id)
                #sys.stderr.write("\nAlleles: %s" % str(alleles[ix]))
                #sys.stderr.write("\n Hap 1: %s, Hap 2: %s\n" % (blockvar.hap1, blockvar.hap2))
                #sys.stderr.flush()
                continue
        if len(allele_state) < 1:
            sys.stderr.write("Warning: no haplotype information in read.\n")
            sys.stderr.write("\nHair read: %s" % read.read_id)
            sys.stderr.write("\nAlleles: %s\n" % str(alleles))
            sys.stderr.flush()
            hap = -1
        print sum(allele_state), len(allele_state)
        if sum(allele_state) < 0:
            hap = 1
        elif sum(allele_state) > 0:
            hap = 2
        else:
            hap = 0
        read.haplotypes[offset] = hap
    return read

'''
interblock_stats()

Usage: Creates a tab-separated values file with statistics about reads overlapping
between nearby blocks, and finds the concordance of these interblock reads
with haplotypes in other blocks. 

inputs:
    hair_reader
        A HairReader object
    block_reader
        A HapcutReader object
    out_stats
        A string where the .tsv should be written. Defaults
        to the hairs filename given in the input + 'interblock_stats.tsv'
outputs:
    none-writes to file directly

'''

def interblock_stats(hair_reader, block_reader, out_stats, bam_fp):
    blockdist = []
    lastChr = None
    lastPos = None
    lastBlock = None
    lastReads = set()
    for k,read in hair_reader.reads.iteritems():
        if read.haplotypes == dict():
            read = greedy_partition(read, block_reader)
    for ix, key in enumerate(sorted(block_reader.blocks.keys())):
        sys.stdout.write('\r%s percent done.' % round(ix/float(len(block_reader.blocks))*100))
        sys.stdout.flush()
        block = block_reader.blocks[key]
        if block.read_set == set():
            block.addReadsToBlock(hair_reader.reads)
        currBlock = block.offset
        currChr = block.chrom
        currPos = block.start
        if lastBlock != None:
            if lastChr == currChr:
                interblock_reads = block.interblock_reads(lastBlock_obj.informative_reads)
                row=[lastBlock, currBlock, currChr, lastPos, currPos, currPos-lastPos,
                     lastBlock_obj.end-lastBlock_obj.start, block.end-block.start,
                     len(lastBlock_obj.variant_ids), len(block.variant_ids), 
                     len(lastBlock_obj.read_set), len(block.read_set),
                     len(list(bam_fp.fetch(region=lastChr + ':' + str(lastBlock_obj.start) + '-' + str(lastBlock_obj.end)))),
                     len(list(bam_fp.fetch(region=lastChr + ':' + str(block.start) + '-' + str(block.end)))),
                     len(interblock_reads),
                     len(list(bam_fp.fetch(region=lastChr + ':' + str(lastBlock_obj.end) + '-' + str(block.start))))]
                     #lastBlock_obj.concordance(lastBlock_obj.informative_reads), 
                     #block.concordance(block.informative_reads)]
                blockdist.append(row)
            else:
                continue
        lastBlock = currBlock
        lastBlock_obj = block
        lastChr = currChr
        lastPos = block.end
    header = ['block1', 'block2', 'chrom', 'block1_end', 'block2_start', 
              'distance', 'block1_size', 'block2_size', 'block1_variants', 'block2_variants', 
              'block1_informative_reads', 'block2_informative_reads', 
              'block1_reads', 'block2_reads',
              'informative_interblock_reads', 
              'all_interblock_reads'] 
              #'block1_concordance', 'block2_concordance']
    info = pd.DataFrame(blockdist, columns=header)
    info.to_csv(out_stats, sep="\t")

    # use freebayes call to modify the reference (reference mask to N)
# look at reads that span more than two SNPs
    # list(bam_fp.fetch(region=lastChr + ':' + str(block.start) + '-' + str(block.end)))
    # see if the SNP was there or not - if it's there but the alignment is messed up we may need to modify it
# compare haplotype calls to hg003 and hg004

def reverse_compl(seq):
    translate = {'A':'T', 
                 'T':'A', 
                 'C':'G',
                 'G':'C'}
    return ''.join([translate[s] for s in seq])

def hamming_dist(str1, str2):
    difference = 0
    for x,y in zip(str1, str2):
        if x != y:
            difference += 1
    return difference

# get aligned portion per region
# IPD data 
def get_matched_bases_in_read(bamread, in_pos):
    bpos = bamread.get_aligned_pairs(matches_only=True)
    positions = [i[0] for i in bpos]
    refpos =  [i[1] for i in bpos] # positions in reference
    refmap = dict(zip(refpos, positions))
    outseq = [bamread.seq[refmap[i]] if i in refmap else 'N' for i in in_pos]
    outseq = ''.join(outseq)
    return outseq, ''

def reverse_compl(seq):
    translate = {'A':'T', 
                 'T':'A', 
                 'C':'G',
                 'G':'C'}
    return ''.join([translate[s] for s in seq])

# we can make this program phase the remaining reads
def unphased_heuristics(hair_reader, block_reader, bam_fp):
    #1. identify unphased reads per block
    unphased_reads = dict()
    uphase_read_set = set()
    #pbar = ProgressBar(len(block_reader.blocks))
    for bid, block in block_reader.blocks.iteritems():
        block.add_unphased_reads(bam_fp)
        pbar2 = ProgressBar(len(block.unphased_reads))
        for bamread in block.unphased_reads:
            pbar2.animate()
            bs, be = bamread.reference_start, bamread.reference_end # 0-based indexing
            br_variants = []
            for variant in block.variants.values():
                if (variant.pos >= bs) & (variant.pos <= be):
                    len_var = len(variant.r_allele)
                    var_id = variant.var_id
                    refcall = variant.r_allele 
                    altcall = variant.v_allele.split(',') # array of possible alternate alleles
                    read_bases, qual_vals = get_matched_bases_in_read(bamread,range(variant.pos-1, variant.pos-1+len_var))
                    # now let's do some primitive variant calling
                    read_call = None # this will be an integer corresponding to the variant call of the read
                    read_dist = [hamming_dist(refcall, read_bases)]
                    for a in altcall:
                        read_dist.append(hamming_dist(a, read_bases))
                    mindist = np.min(read_dist)
                    min_allele = [i for i, x in enumerate(read_dist) if x == mindist]
                    if len(min_allele) == 1:
                        read_call = min_allele[0]
                    # if we got a variant call, let's add it to the array
                    if read_call != None:
                        br_variants.append([var_id, read_call, qual_vals])
            # end variant loop
            if len(br_variants) > 0:
                # let's print a hairreader line
                ranges = []
                qline = ''
                for k, g in groupby(enumerate(br_variants), lambda (i,x):i-x[0]):
                    group = map(itemgetter(1), g)
                    ranges.append((group[0][0], group))
                hairline = [str(len(ranges)), bamread.query_name]
                for gid, group in ranges:
                    hairline.append(str(gid))
                    gline = ''
                    for g in group:
                        gline += str(g[1])
                        qline += g[2]
                    hairline.append(gline)
                hairline.append(qline)
                hairline = ' '.join(hairline)
                '''this now exactly matches the hairs file format'''
                # print hairline 
                if bamread.query_name not in uphase_read_set:
                    hairread = HapCutRead(hairline)
                    hairread.chrom = block.chrom
                    hairread.start = bs
                    hairread.end = be
                    hairread.blockcount = 1
                    hairread.blocks = [[(pos,str(allele)) for pos, allele, qual in br_variants]]
                    unphased_reads[bamread.query_name] = hairread
                    uphase_read_set.add(bamread.query_name)
                else:
                    other_read = unphased_reads[bamread.query_name]
                    other_read.blockcount += 1
                    other_read.blocks.append([(pos,str(allele)) for pos, allele, qual in br_variants])
                    other_read.positions.extend([pos for pos, allele, qual in br_variants])
                    other_read.alleles.extend([str(allele) for pos, allele, qual in br_variants])
            # end check for variants in bamread
            # break
        # end bamread loop
        # break
    # end block loop
    return unphased_reads

# run the program if called from the command line
#if __name__ == "__main__":
#   main(sys.argv[1:])


In [142]:
basedir = '/sc/orga/scratch/bashia02/collaborations/hardik_shah/jason_new/hapcut_outputs/hg002_re_000000F/'
bam_fp = pysam.Samfile(basedir + 'hg002_000000F.new.merged.bam', 'rb')
out_fp = pysam.AlignmentFile(basedir +'hg002.000000F.het_sites_truth_partition_snps_only.bam', 'wb', template=bam_fp)

def truth_partition(bam_fp, het_sites, out_fp):
    pbar = ProgressBar(450000)
    br_variants = []
    reads_written = set()
    for k, line in het_sites.iterrows():
        chrom, pos, ref, alt, hg002_genotype = line
        pos = int(pos)
        for read in bam_fp.fetch(reference=chrom, start=pos, end=pos+1):
            read_vars = []
            read_hap = 0
            if read.qname in reads_written:
                continue
            else:
                reads_written.add(read.qname)
            pbar.animate()
            readstart, readend = read.reference_start, read.reference_end
            # determine all sites read overlaps
            read_sites = het_sites[(het_sites['pos'] >= readstart) & (het_sites['pos'] <= readend)]
            for ix, site in read_sites.iterrows():
                vals = [len(i) for i in site[3]]
                vals.append(len(site[2]))
                len_var = np.min(vals)
                sitepos = site[1]
                refcall = site[2] 
                altcall = site[3] # array of possible alternate alleles
                read_bases, qual_vals = get_matched_bases_in_read(read,range(sitepos-1, sitepos-1+len_var))
                # now let's do some primitive variant calling
                read_call = None # this will be an integer corresponding to the variant call of the read
                read_dist = [hamming_dist(refcall, read_bases)]
                for a in altcall:
                    read_dist.append(hamming_dist(a, read_bases))
                mindist = np.min(read_dist)
                min_allele = [i for i, x in enumerate(read_dist) if x == mindist]
                if len(min_allele) == 1:
                    read_call = min_allele[0]
                # if we got a variant call, let's add it to the array
                if read_call != None:
                    hap = None
                    mat, pat = [int(a) for a in hg002_genotype.split("|")]
                    if read_call == mat:
                        hap = -1
                    if read_call == pat:
                        hap = 1
                    br_variants.append([chrom, pos, ref, alt, read_bases, read_call, hap])
                    read_vars.append(hap)
            if len(read_vars) > 0:
                if sum(read_vars) < 0:
                    read_hap = 1
                elif sum(read_vars) > 0:
                    read_hap = 2
                else:
                    read_hap = 0
                haptag = [("ZH", read_hap), ("ZB", 'truth'), ("ZV", len(read_vars)), ('ZS', abs(sum(read_vars)))]
                read.tags += haptag
                out_fp.write(read)
    return br_variants


In [162]:
def describe(block):
    for i in block.__dict__.keys():
        item = getattr(block, i)
        ctype = type(getattr(block, i))
        print i, ctype
        if (ctype == type(0)) | (ctype == type(np.int64(0))):
            print '\t', item
        elif ctype == type({}):
            print '\t', item.keys()[0:1], '...'
        elif ctype == type([]):
            print '\t', item[0:1], '...'
        elif (ctype == type(set())) | (ctype == type(frozenset())):
            print '\t', list(item)[0:1], '...'

def updateTranslate(block_reader):
    translate = block_reader.translate
    for bid, block in block_reader.blocks.iteritems():
        for v in block.variant_ids:
            translate[v] = block.offset
    return translate

def addBlocks(block, block2, b1hap, b2hap):
    '''
    self.offset      = int(ll[2])
    self.total_len   = int(ll[4])
    self.phased      = int(ll[6])
    self.span        = int(ll[8])
    self.MECscore    = float(ll[10])
    self.fragments   = int(ll[12])
    self.variants 	 = dict() # default to empty
    self.variant_ids = set()
    self.chrom = 0
    self.start = 0
    self.end = 0
    self.informative_reads = []
    self.unphased_reads = []
    self.unphased_read_set = set()
    self.read_count = 0
    self.read_set = set()
    '''
    if block.offset == block2.offset:
        sys.stderr.write("Warning: blocks already merged.")
        sys.stderr.flush()
        return 1
    if block.chrom != block2.chrom:
        raise AwesomeError("Can't add blocks on different contigs.")
    block.offset = block.offset if block2.offset > block.offset else block2.offset
    block.MECscore += block2.MECscore
    block.end = block2.end if block2.end > block.end else block.end
    block.start = block2.start if block2.start < block.start else block.start
    block.fragments += block2.fragments
    block.informative_reads.extend(block2.informative_reads)
    block.read_count += block2.read_count
    block.total_len += block2.total_len
    block.span = block.end - block.start
    block.variant_ids = block.variant_ids.union(block2.variant_ids)
    block.read_set = block.read_set.union(block2.read_set)
    block.unphased_reads.extend(block2.unphased_reads)
    block.phased += block2.phased
    block.unphased_read_set = block.unphased_read_set.union(block2.unphased_read_set)
    
    for v in block2.variants:
        haps = v.hap1, v.hap2
        v.hap1 = haps[1] if b1hap != b2hap else haps[0]
        v.hap2 = haps[0] if b1hap != b2hap else haps[1]
        
    return 0

# run code

In [32]:
basedir = '/sc/orga/scratch/bashia02/collaborations/hardik_shah/jason_new/hapcut_outputs/hg002_re_000000F/'
bam_fp = pysam.Samfile(basedir + 'hg002_000000F.new.merged.bam', 'rb')
out_fp = pysam.AlignmentFile(basedir +'ipds_fwd/test_sig_denovo.bam', 'wb', template=bam_fp)
hairs_file = basedir + 'ipds_fwd/test_sig_denovo'
hapcut_file = basedir + 'ipds_fwd/test_hapcut_sig_denovo.hapcut'

hair_reader = HairReader(hairs_file)
block_reader = HapCutReader(hapcut_file)

In [ ]:
hair_reader.assemble_blocks(block_reader)
unphased_reads = unphased_heuristics(hair_reader, block_reader, bam_fp)
stats_file = basedir + 'hapcut_qv13_mq10/hg002.000000F.qv13mq10.redo.merged.iblockstats.tsv'
tag_reads(bam_fp, hair_reader, block_reader, unphased_reads, out_fp) #begin tagging reads
interblock_stats(hair_reader, block_reader, stats_file, bam_fp) #generate interblock stats
out_fp.close() #IMPORTANT

import pickle
pickle.dump(unphased_reads, open('hg002_more_hapcut_unphased_reads.json', 'wb'))

In [143]:
variants = truth_partition(bam_fp, het_sites, out_fp)
out_fp.close()

[**************   37%                  ]  166898 of 450000 complete Elapsed: 202.2 minutes	 Remaining: 343.0 minutes


In [141]:
vcf_truth = ht.VCF_Reader("/hpc/users/neffr01/jason_new/trio_analysis/aj_trio_phasebytrans.phased-trio.snps.000000F.vcf")
vcf_truth.parse_meta()
vcf_truth.make_info_dict()
het_sites = [] # an array of het sites that we will read into mem
pbar = ProgressBar(130000)
for var in vcf_truth:
    pbar.animate()
    hg002_genotype = var.samples['hg002']['GT']
    if var.chrom != "000000F":
        print "Done"
        break
    if '|' in hg002_genotype:
        if len(set(hg002_genotype.split("|"))) == 2:
            het_sites.append([var.chrom, var.pos.pos, var.ref, var.alt, hg002_genotype])
het_sites = pd.DataFrame(het_sites, columns=["chrom", "pos", "ref", "alt", "genotype"])

[******           17%                  ]  22157 of 130000 complete Elapsed: 0.0 minutes	 Remaining: 0.1 minutes


##other stuff

In [107]:
len(unphased_reads)

71313

In [143]:
block_phaseable2 = block_phaseable

In [5]:
block_phaseable = []
block_index = sorted(block_reader.blocks.keys())
block = block_reader.loc(block_index[0])
block.addReadsToBlock(dict(unphased_reads, **hair_reader.reads))

for ix, bid in enumerate(block_index[1:len(block_index)]):
    block = block_reader.loc(bid)
    block.addReadsToBlock(hair_reader.reads)
    hairreads = block.read_set
    block.addReadsToBlock(dict(unphased_reads, **hair_reader.reads))
    block.add_unphased_reads(bam_fp)
    allphaseable = block.read_set
    block.unphased_read_set = unphasedreads.difference(allphaseable)
    block.unphased_reads = {x: block.unphased_reads[x] for x in block.unphased_reads if x in block.unphased_read_set}
    print bid, len(hairreads), len(block.read_set.difference(hairreads)), len(block.read_set), len(block.unphased_read_set)
    print len(block.interblock_reads(block_reader.loc(block_index[ix-1]).informative_reads))
    block_phaseable.append([bid, len(hairreads), 
                            len(block.read_set.difference(hairreads)), 
                            len(block.read_set), len(block.unphased_read_set)])

NameError: name 'unphasedreads' is not defined

In [16]:
unphased_read_set = frozenset(unphased_reads.keys())

In [26]:
for ix, bid in enumerate(block_index):
    block = block_reader.loc(bid)
    if len(block.unphased_read_set.intersection(block.read_set)) == 0:
        block.addReadsToBlock(dict(unphased_reads, **hair_reader.reads))
    print ix, bid, len(block.read_set.difference(unphased_read_set)), len(block.read_set), len(block.unphased_read_set.difference(block.read_set)), len(block.read_set.union(block.unphased_read_set))
    sys.stdout.flush()

0 4 2609 4936 4143 9079
1 1990 15 72 116 188
2 2036 88 341 597 938
3 2282 100 270 413 683
4 2639 78 202 396 598
5 2823 829 2380 1499 3879
6 4032 3856 7044 1727 8771
7 7395 249 816 585 1401
8 7980 3 59 73 132
9 8031 41 198 210 408
10 8171 3085 5944 1078 7022
11 10922 922 1780 548 2328
12 11744 9 58 149 207
13 11799 13 62 268 330
14 11852 78 209 236 445
15 11991 145 421 663 1084
16 12493 2665 5069 1447 6516
17 15379 4761 8725 2211 10936
18 19238 2207 3941 2478 6419
19 21841 1 68 64 132
20 21867 162 559 461 1020
21 22338 2295 4521 1640 6161
22 24817 3856 6606 1498 8104
23 28033 1304 2147 1510 3657
24 29447 144 542 583 1125
25 29941 1 55 110 165
26 30019 14 60 62 122
27 30052 65 388 270 658
28 30341 27 125 284 409
29 30414 6 66 84 150
30 30444 4 4 83 87
31 30471 2543 4291 1232 5523
32 32711 47 265 229 494
33 32834 2375 4146 1774 5920
34 35631 101 299 177 476
35 35855 62 184 43 227
36 36028 358 1012 985 1997
37 36929 12 63 31 94
38 36958 50 51 164 215
39 37086 159 598 377 975
40 37528 1302 

In [142]:
pd.DataFrame(block_phaseable).to_csv('/hpc/users/neffr01/jason_new/hapcut_outputs/hg002_re_000000F/hapcut_qv1_mq1/block_phaseable.csv', sep='\t')

In [49]:
linked_list = []
block_list = sorted(block_reader.blocks.keys())
def simple_func(name1, arr, cutoff, block_list):
    outarr = []
    block1 = name1[0]
    if name1[1] == 0:
        line = None
        outarr.append(line)
    for block, b_arr in groupby(arr, lambda x: x[0]):
        b_arr = list(b_arr)
        bname = list(b_arr)[0]
        block2 = bname[0]
        if bname[1] == 0:
            continue
        if abs(block_list.index(block1) - block_list.index(block2)) > 1:
            continue
        b_arr = [b[1] for b in b_arr]
        for name, group in groupby(b_arr):
            line = []
            group = list(group)
            if len(group)/float(len(b_arr)) > cutoff:
                b1, b2 = block_reader.loc(block1), block_reader.loc(block2)
                inbd, inbd_start, inbd_end = 0,0,0
                if block2 > block1:
                    inbd = b2.start - b1.end
                    inbd_start, inbd_end = b1.end, b2.start
                else:
                    inbd = b1.start - b2.end
                    inbd_start, inbd_end = b2.end, b1.start
                line = [name1, bname, len(group)/float(len(b_arr)),
                        len(b_arr), b1.end-b1.start, b2.end-b2.start, inbd, inbd_start, inbd_end]
            else:
                line = None
            outarr.append(line)
    return outarr
            
interblock = []
for k, read in unphased_reads.iteritems():
    if read.haplotypes == dict():
        read = greedy_partition(read, block_reader)
    if len(read.haplotypes) > 1:
        interblock.append([(a,b) for a,b in read.haplotypes.iteritems()])
interblock = sorted(interblock, key=lambda x: x[0])
for name, group in groupby(interblock, lambda x: x[0]):
    outarr = simple_func(name, [b[1] for b in list(group)], .5, block_list)
    for o in outarr:
        if o != None:
            linked_list.append(o)
print len(linked_list)
for l in linked_list:
    print l

53
[(4032, 1), (2823, 1), 1.0, 1, 744157, 310402, 9927, 1091023, 1100950]
[(7980, 1), (8031, 1), 1.0, 11, 5930, 29777, 4110, 1982896, 1987006]
[(11744, 1), (10922, 2), 1.0, 1, 12425, 185792, 15903, 2787419, 2803322]
[(11852, 2), (11799, 1), 1.0, 1, 25303, 19767, 7702, 2847178, 2854880]
[(15379, 1), (19238, 1), 1.0, 2, 938799, 575875, 12030, 4495257, 4507287]
[(21841, 2), (19238, 2), 0.5714285714285714, 7, 3861, 575875, 9919, 5083162, 5093081]
[(28033, 1), (29447, 2), 1.0, 2, 332197, 95699, 6130, 6780299, 6786429]
[(28033, 2), (29447, 2), 1.0, 9, 332197, 95699, 6130, 6780299, 6786429]
[(29941, 1), (29447, 1), 1.0, 1, 8851, 95699, 6966, 6882128, 6889094]
[(29941, 2), (29447, 1), 1.0, 2, 8851, 95699, 6966, 6882128, 6889094]
[(32834, 0), (35631, 2), 1.0, 4, 580773, 32198, 4707, 8202797, 8207504]
[(32834, 0), (35631, 2), 1.0, 1, 580773, 32198, 4707, 8202797, 8207504]
[(32834, 1), (35631, 1), 1.0, 2, 580773, 32198, 4707, 8202797, 8207504]
[(37528, 0), (37086, 2), 1.0, 1, 265526, 78357, 5587,

In [97]:
linker_dict = dict()
for row in linked_list:
    b1, b2 = row[0], row[1]
    support = row[2] * row[3] 
    if b1 > b2:
        tmp = b1
        b1 = b2
        b2 = tmp
    if (b1[1] == 0) | (b2[1] == 0):
        continue
    if linker_dict.has_key(b1[0]):
        [bb1, bb2, bsup] = linker_dict[b1[0]]
        if bsup < support:
            linker_dict[b1[0]] = [b1, b2, support]
    else:
        linker_dict[b1[0]] = [b1, b2, support]

In [180]:
block_reader.translate = updateTranslate(block_reader)

In [181]:
for k, row in linker_dict.iteritems():
    block = block_reader.loc(row[1][0])
    b1hap = row[0][1]
    b2hap = row[1][1]
    offset = row[1][0]
    for var_id, v in block.variants.iteritems():
        if var_id < offset:
            continue
        haps = v.hap1, v.hap2
        v.hap1 = haps[1] if b1hap != b2hap else haps[0]
        v.hap2 = haps[0] if b1hap != b2hap else haps[1]

In [174]:
for k,row in linker_dict.iteritems():
    b1 = block_reader.loc(row[0][0])
    b2 = block_reader.loc(row[1][0])
    print "Preparing to merge:", b1.offset, b2.offset, row[2]
    sys.stdout.flush
    confirm = raw_input("Are you sure? This cannot be undone. (y/n)")
    if confirm == 'y':
        a = addBlocks(b1, b2)
        block_reader.translate = updateTranslate(block_reader)
        if a == 0:
            del block_reader.blocks[row[1][0]]
    else:
        break

Preparing to merge: 28033 28033 9.0
Are you sure? This cannot be undone. (y/n)y


Preparing to merge: 62596 62596 1.0
Are you sure? This cannot be undone. (y/n)y


Preparing to merge: 2823 2823 1.0
Are you sure? This cannot be undone. (y/n)y


Preparing to merge: 15379 15379 2.0
Are you sure? This cannot be undone. (y/n)y


Preparing to merge: 11799 11799 1.0
Are you sure? This cannot be undone. (y/n)y


...

Preparing to merge: 120476 120476 1.0
Are you sure? This cannot be undone. (y/n)y


Preparing to merge: 106269 106269 12.0
Are you sure? This cannot be undone. (y/n)y


Preparing to merge: 130462 130462 1.0
Are you sure? This cannot be undone. (y/n)y


Preparing to merge: 67618 67618 10.0
Are you sure? This cannot be undone. (y/n)y


Preparing to merge: 15379 21841 4.0
Are you sure? This cannot be undone. (y/n)y
Preparing to merge: 10922 11744 1.0
Are you sure? This cannot be undone. (y/n)y
Preparing to merge: 28033 29941 2.0
Are you sure? This cannot be undone. (y/n)y
Preparing to merge: 7980 8031 11.0
Are you sure? This cannot be undone. (y/n)y
Preparing to merge: 64173 64483 4.0
Are you sure? This cannot be undone. (y/n)y
Preparing to merge: 64057 64081 9.0
Are you sure? This cannot be undone. (y/n)y
Preparing to merge: 130748 130916 1.0
Are you sure? This cannot be undone. (y/n)y
Preparing to merge: 32834 35631 2.0
Are you sure? This cannot be undone. (y/n)y
Preparing to merge: 116556 118718 7.0
Are you sure? This cannot be undone. (y/n)y
Preparing to merge: 62596 63220 1.0
Are you sure? This cannot be undone. (y/n)y
Preparing to merge: 105933 106054 1.0
Are you sure? This cannot be undone. (y/n)y
Preparing to merge: 39761 39788 5.0
Are you sure? This cannot be undone. (y/n)y
Preparing to merge: 119609 120115 1

In [177]:
merged_stats = '/hpc/users/neffr01/jason_new/hapcut_outputs/hg002_re_000000F/hapcut_with_indels/hg002_000000F.indels.aftermerge.interblock_stats.tsv'
interblock_stats(hair_reader, block_reader, merged_stats, bam_fp) #generate interblock stats

99.0 percent done.

In [55]:
out_fp.close()

In [172]:
print len(unphased_reads)
print len(hair_reader.reads)

71313
162094


In [53]:
%matplotlib inline
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
plt.style.use('ggplot')
rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 150
rcParams['font.size'] = 16
rcParams['font.family'] = 'Bitstream Vera Sans'

In [22]:
unphased_read_vc = [len(read.alleles) for read in unphased_reads.values()]
inform_read_vc = [len(read.alleles) for read in hair_reader.reads.values()]
pd.DataFrame(zip(unphased_read_vc, inform_read_vc)).to_csv('/hpc/users/neffr01/jason_new/hapcut_outputs/hg002_re_000000F/hapcut_with_indels/unphased_vs_inform_varcount-aftermerge.tsv', sep='\t')
plt.hist(inform_read_vc, bins=range(0,20), color='b', alpha=0.5, label="informative reads")
plt.hist(unphased_read_vc, bins=range(0,20), color='r', alpha=0.5, label="phaseable reads")
plt.legend()
plt.title('Phasing between read types')
plt.xlabel('Variants per read')
plt.ylabel('Reads')
plt.savefig("/hpc/users/neffr01/jason_new/hapcut_outputs/hg002_re_000000F/hapcut_with_indels/varcount_per_read_withindels-aftermerge.png")
plt.show()

IOError: [Errno 13] Permission denied: '/hpc/users/neffr01/jason_new/hapcut_outputs/hg002_re_000000F/hapcut_with_indels/unphased_vs_inform_varcount-aftermerge.tsv'

In [215]:
for k, block in block_reader.blocks.iteritems():
    block.read_set = set(block.read_set)
for k, read in unphased_reads.iteritems():
    read.assemble_blocks(block_reader)
    if read.blockcount > 1:
        if read.end-read.start > 60000:
            continue
        for rb in read.blocks:
            sys.stdout.write(str(rb[0][0]) + '\t')
        sys.stdout.write('\t'.join([str(len(read.alleles)), str(read.start), str(read.end)]))
        sys.stdout.write('\n')

9918	9921	2	13349960	13363167
2010	2007	2	3021394	3035106
15765	15766	2	21552954	21567673
15765	15766	2	21552954	21567673
12333	12328	3	17178259	17196788


In [21]:
for readblock in read.blocks:
    print readblock
    print block_reader.loc(readblock[0][0])

[(9010, '0'), (9012, '0')]
<Block, offset_id: 8516>
[(9013, '0'), (9014, '0')]
<Block, offset_id: 8516>


In [14]:
b = block_reader.loc(1138)
for k,read in unphased_reads.iteritems():
    if read.blockcount > 1:
        read = greedy_partition(read, block_reader)
        print read.haplotype_fields()
        break

[('ZH', '8516,1'), ('ZB', 2), ('ZV', 4)]
